In [1]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
'''from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "TheBloke/openchat-3.5-0106-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=False, revision='main')
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0)
hf = HuggingFacePipeline(pipeline=pipe)
above very slow. ollama is goated + easier to test
'''

'from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline\nfrom transformers import AutoModelForCausalLM, AutoTokenizer, pipeline\n\nmodel_id = "TheBloke/openchat-3.5-0106-GPTQ"\ntokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)\nmodel = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=False, revision=\'main\')\npipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0)\nhf = HuggingFacePipeline(pipeline=pipe)\nabove very slow. ollama is goated + easier to test\n'

In [3]:
'''
To run this you need to install ollama first 
https://ollama.com/download
Run ollama after installation (it wont show up at all as it runs in the background)
In terminal of this venv run "ollama pull openchat" to use this model. 
Don't default pip install torch unless you want to watch paint dry. Use the gpu specific install 
'''

local_llm = "openchat"
#https://ollama.com/library?sort=popular

In [4]:
import torch

torch.cuda.empty_cache()

In [5]:
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant

loader = DirectoryLoader('C:/Users/skfrqt/Desktop/jk/mil6/docs/aws-documentation-main', glob="**/*.pdf", show_progress=True,loader_cls=UnstructuredFileLoader, recursive=True)

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

100%|██████████| 1/1 [00:31<00:00, 31.04s/it]


In [ ]:
#langchain bastards have progress bar class for every other embedding but not fastembed (⩺_⩹)
qdrant_store = Qdrant.from_documents(
    location=":memory:", #temp memory im just using for testing purposes. not suitable for production but fine enough for demo
    collection_name="AWS_TEST",
    documents=texts,
    embedding=embeddings
)
retriever = qdrant_store.as_retriever()

In [23]:

# Define roles
roles = ['roleX', 'roleY', 'roleZ', 'admin']
role_files = {
    'roleX': 'C:/Users/skfrqt/Desktop/jk/mil6/docs/aws-documentation-main', #change the path to files for a speciefic role
    'roleY': 'C:/Users/skfrqt/Desktop/jk/mil6/docs/aws-documentation-main', #change the path to files for a speciefic role
    'roleZ': 'C:/Users/skfrqt/Desktop/jk/mil6/docs/aws-documentation-main', #change the path to files for a speciefic role
    'admin': 'C:/Users/skfrqt/Desktop/jk/mil6/docs/aws-documentation-main', #change the path to files for a speciefic role
}

# Load docs for each role to Qdrant
for role, files in role_files.items():
    loader = DirectoryLoader('C:/Users/skfrqt/Desktop/jk/mil6/docs/aws-documentation-main', glob="**/*.pdf", show_progress=True,loader_cls=UnstructuredFileLoader, recursive=True)
    documents = loader.load()
    texts = text_splitter.split_documents(documents)
    qdrant_store = Qdrant.from_documents(
        location=":memory:",
        collection_name=role,
        documents=texts,
        embedding=embeddings
    )

# Ask user his role
user_role = input('What role are you: ' + ', '.join(roles) + '\n')

# Check if role exists
if user_role not in roles:
    print('Invalid role. Please try again.')
else:
    # create retriever for user
    retriever = qdrant_store.as_retriever(collection_name=user_role)

100%|██████████| 1/1 [00:27<00:00, 27.11s/it]


In [7]:
from typing import Dict, TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [8]:
from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

### Nodes ###

#TODO: Use multiple models for different tasks. For example, use a model that is good at generating questions for the transform_query node. Will reduce inference time and improve performance
#https://ollama.com/library?sort=popular



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    # print("---RETRIEVE---")
    # state_dict = state["keys"]
    # question = state_dict["question"]
    # documents = retriever.get_relevant_documents(question)
    # return {"keys": {"documents": documents, "question": question}}
#   change the retrieve func 
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = retriever.get_relevant_documents(question)
    return {"keys": {"documents": documents, "question": question}}



def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    
    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    llm = ChatOllama(model=local_llm, temperature=0)

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {"documents": documents, "question": question, "generation": generation}
    }

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant documents
    """

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    # LLM
    llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keywords related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
        input_variables=["question","context"],
    )

    # Chain
    chain = prompt | llm | JsonOutputParser()

    # Score
    filtered_docs = []
    for d in documents:
        score = chain.invoke(
            {
                "question": question,
                "context": d.page_content,
            }
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue

    return {"keys": {"documents": filtered_docs, "question": question}}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    # LLM
    llm = ChatOllama(model=local_llm, temperature=0)
    
    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval. \n 
        Look at the input and try to reason about the underlying sematic intent / meaning. \n 
        Here is the initial question:
        \n ------- \n
        {question} 
        \n ------- \n
        Formulate an improved question:""",
        input_variables=["question"],
    )

    # Chain
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    return {"keys": {"documents": documents, "question": better_question}}

def prepare_for_final_grade(state):
    """
    Passthrough state for final grade.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): The current graph state
    """

    print("---FINAL GRADE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    generation = state_dict["generation"]

    return {
        "keys": {"documents": documents, "question": question, "generation": generation}
    }


### Edges ###

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents(state):
    """
    Determines whether the generation is grounded in the document.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Binary decision
    """
    print("---GRADE GENERATION vs DOCUMENTS---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    generation = state_dict["generation"]

    # LLM
    llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing whether an answer is grounded in / supported by a set of facts. \n 
        Here are the facts:
        \n ------- \n
        {documents} 
        \n ------- \n
        Here is the answer: {generation}
        Give a binary score 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts. \n
        Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
        input_variables=["generation", "documents"],
    )

    # Chain
    chain = prompt | llm | JsonOutputParser()
    score = chain.invoke({"generation": generation, "documents": documents})
    grade = score["score"]

    if grade == "yes":
        print("---DECISION: SUPPORTED, MOVE TO FINAL GRADE---")
        return "supported"
    else:
        print("---DECISION: NOT SUPPORTED, GENERATE AGAIN---")
        return "not supported"

def grade_generation_v_question(state):
    """
    Determines whether the generation addresses the question.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Binary decision
    """

    print("---GRADE GENERATION vs QUESTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    generation = state_dict["generation"]

    llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
        Here is the answer:
        \n ------- \n
        {generation} 
        \n ------- \n
        Here is the question: {question}
        Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
        Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
        input_variables=["generation", "question"],
    )

    # Prompt
    chain = prompt | llm | JsonOutputParser()
    score = chain.invoke({"generation": generation, "question": question})
    grade = score["score"]

    if grade == "yes":
        print("---DECISION: USEFUL---")
        return "useful"
    else:
        print("---DECISION: NOT USEFUL---")
        return "not useful"

In [14]:
import pprint
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("prepare_for_final_grade", prepare_for_final_grade)  # passthrough

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents,
    {
        "supported": "prepare_for_final_grade",
        "not supported": "generate",
    },
)
workflow.add_conditional_edges(
    "prepare_for_final_grade",
    grade_generation_v_question,
    {
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [17]:
inputs = {"keys": {"question": "Is Amazon EBS encryption available on M3 instances?"}}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value['keys']['generation'])  

---RETRIEVE---
"Node 'retrieve':"
{ 'documents': [ Document(page_content='iv\n\nAmazon Pinpoint Developer Guide', metadata={'source': 'C:\\Users\\skfrqt\\Desktop\\jk\\mil6\\docs\\aws-documentation-main\\documents\\amazon-pinpoint-developer-guide\\doc-source\\pinpoint-dg.pdf', '_id': '18a91f0f4a6a4ea68803ac099dca9563', '_collection_name': 'AWS_TEST'}),
                 Document(page_content='"ses:List*", "sns:ListTopics", "ses:Describe*", "s3:List*" ], "Resource": "*" } ] }\n\nIn the preceding policy example, replace region with the name of an AWS Region, replace accountId with your AWS account ID, and replace projectId with the ID of the Amazon Pinpoint project that you want to provide access to.\n\nAmazon Pinpoint SMS and Voice API Actions\n\nAdministrator Access\n\nThe following policy grants full access to the Amazon Pinpoint SMS and Voice API:\n\n{ "Version": "2018-09-05", "Statement": [ { "Action": [ "sms-voice:*" ], "Effect": "Allow", "Resource": "*" } ] }\n\nRead-Only Access\n\n

GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limitby setting the `recursion_limit` config key.

In [13]:
'''
Notes:
The QA csv sheet provided has some errors in it (e.g. the question "What is the maximum number of security groups associated with a load balancer?" is 5 but the wrong answer expected is 50
Llamaindex/langchain huggingfacellm wrapper literally makes inference almost 7x slower vs using a direct pipeline.
Ollama is godsend but we should use it with linux/wsl for faster inference 
Although llamaindex does have some nice features that langchain doesnt have (at a glance) - we should use both of them in production instead of only one. both of their docs arent very clear on lots of things/are outdated
I havent tested performance between chroma and qdrant but at a glance qdrant seems to be faster
Above code is from langchain's example (wouldve saved a months worth of work if we pivoted earlier) - wasted an entire day trying to use with huggingface but decided on ollama like their docs 
Performance completely depends on the question asked. If the question is too vague, the model will take a long time to generate a response as it has to make multiple calls to rewrite question/rank response to prevent hallucinations. 
Current benchmark is 25-45s on average. 2 min slowest i've seen. Needs some more tuning - will see if using a different model for each task will help
Just generating an answer on its own without going through workflow takes a couple of seconds

(Modular workflow graph - needs some further tuning)
Step 1: User asks a question
Step 2: Retrieve relevant document
Step 3: Grade document relevance
Step 4: If document is not relevant go back to step 2 (error here is if all documents are not relevant we get stuck in an "infinite" loop but it usually stops after 25 calls - roughly 2 mins)
Step 5: Generate answer
Step 6: Grade answer. If answer is hallucinated/doesnt match document go back to step 5 (same error as 4 if context doesnt exist)
Step 7: If answer is useful, end. If not go back to step 2

'''

#TODO: Use linux/wsl with vLLM for faster inference. Should be almost 2x faster than Windows from some readings?
#TODO: Use llamaindex for some tasks and langchain for others. Both have their own strengths and weaknesses. Both have horrible docs
#TODO: Host qdrant on a server
#TODO: Figure out metadata for documents. Useful for filtering out irrelevant/old answers - should sped up & reduce hallucinations
#TODO: Add preprocessing & postprocessing to the workflow. Will be useful for filtering out irrelevant/old answers - should sped up & reduce hallucinations
#TODO: Test CRAG performance against current self-rag implementation
#TODO: Add chat history for multiple-shot reasoning


'\nNotes:\nThe QA csv sheet provided has some errors in it (e.g. the question "What is the maximum number of security groups associated with a load balancer?" is 5 but the wrong answer expected is 50\nLlamaindex/langchain huggingfacellm wrapper literally makes inference almost 7x slower vs using a direct pipeline.\nOllama is godsend but we should use it with linux/wsl for faster inference \nAlthough llamaindex does have some nice features that langchain doesnt have (at a glance) - we should use both of them in production instead of only one. both of their docs arent very clear on lots of things/are outdated\nI havent tested performance between chroma and qdrant but at a glance qdrant seems to be faster\nAbove code is from langchain\'s example (wouldve saved a months worth of work if we pivoted earlier) - wasted an entire day trying to use with huggingface but decided on ollama like their docs \nPerformance completely depends on the question asked. If the question is too vague, the mode